# Getting Started with Azure Open AI

Based on https://docs.llamaindex.ai/en/stable/getting_started/starter_example/

LlamaIndex is available in Python (these docs) and Typescript. If you're not sure where to start, we recommend reading how to read these docs which will point you to the right place based on your experience level.

## 30 second quickstart

Set an environment variable called OPENAI_API_KEY with an OpenAI API key. 

Install the Python library:

```bash
pip install llama-index-llms-azure-openai
pip install llama-index
```

## Get environment variables

OPENAI_API_VERSION: set this to 2023-07-01-preview This may change in the future.
AZURE_OPENAI_ENDPOINT: your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
OPENAI_API_KEY: your API key

In [1]:
import os

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_API_VERSION = "2023-07-01-preview"


Create a LlamaIndex llm object using AzureOpenAI and query it

In [2]:
from llama_index.llms.azure_openai import AzureOpenAI

llm = AzureOpenAI(
    engine=AZURE_OPENAI_MODEL,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model=AZURE_OPENAI_MODEL,
    api_version=AZURE_OPENAI_API_VERSION,
    temperature=0.0,
)

response = llm.complete("The sky is a beautiful blue and")
print(response)


the sun is shining brightly. The warmth of the sun's rays gently caresses the skin, providing a comfortable and soothing sensation. Fluffy white clouds drift lazily across the vast expanse, creating an ever-changing tapestry that captures the imagination. The light breeze carries with it the fresh scent of nature, perhaps the fragrance of blooming flowers or the clean smell of a recent rain. The sound of birds singing melodically in the distance adds to the serene atmosphere, creating a sense of peace and tranquility. It's the kind of day that invites outdoor activities, from a leisurely walk in the park to a spirited game of frisbee, or simply lying on the grass and watching the clouds roll by. The beautiful blue sky is a classic symbol of hope and possibility, reminding us of the beauty that exists in the world and encouraging us to take a moment to appreciate the simple joys of life.


Create or retrieve the index

In [3]:
import os
from llama_index.core import (SimpleDirectoryReader, VectorStoreIndex)
from llama_index.core import load_index_from_storage, StorageContext

def create_index(persist_dir):
    print("Creating index from data")
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=persist_dir)
    return index

def load_index(persist_dir):
    print("Loading index from storage")
    storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
    index = load_index_from_storage(storage_context)
    return index

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    index = create_index()
else:
    index = load_index(PERSIST_DIR)

query_engine = index.as_query_engine()

Loading index from storage


Create class to instantiate a custom query engine which uses AzureOpenAI as llm provider

In [4]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import PromptTemplate

qa_prompt = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)

class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: AzureOpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)

        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)


Instantiate the query engine

In [5]:
retriever = index.as_retriever()
synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)



Query the LlamaIndex using Azure Open AI

In [6]:
response = query_engine.query("What truth is universally understood?")
print(response)

The universally understood truth is that a single man in possession of a good fortune must be in want of a wife.


Turn on logging

In [7]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Store the model for later retrieval

By default, the data you just loaded is stored in memory as a series of vector embeddings. You can save time (and requests to OpenAI) by saving the embeddings to disk. That can be done with this line:

In [8]:
index.storage_context.persist()